In [34]:
import ezdxf

fname = "../dxffiles/P-E-13,5-BS-Master-09.dxf"
d = ezdxf.readfile(fname)
print("layerspresent:", set(e.dxf.layer for e in d.entities))


layerspresent: {'BSF-CUT-BOTH', 'BSR-CUT-EVO', 'BSM-CUT-BOTH', 'BSM-PLOT-EVO', 'BSR-PLOT-PUMA', 'PATCH-OUTSIDE', 'PATCH-MYLAR', 'BSR-CUT-PUMA', 'CONSTRUCTION', 'BSF-PLOT-PUMA', 'BSM-PLOT-PUMA', 'SPLINES-MASTER', 'PATCH-INSIDE', 'BSF-PLOT-EVO', 'PATCH-REF', 'BSR-PLOT-EVO', 'PUMA-REF', 'HISTORY'}


In [35]:
keeplayers = {"BSF-CUT-BOTH":"1", 'BSF-PLOT-PUMA':"8"}
selectedname = "BSF-PUMA"

doc = ezdxf.new('R12')
block = doc.blocks.new(name=selectedname)

aamalayers = { "1":doc.layers.new("1"), "8":doc.layers.new("8") }
aamalayers["1"].color = 1
aamalayers["8"].color = 3


In [36]:
aamalayers["1"].dxf.name

'1'

In [37]:
NONURBS = True
cuts = [ ]
for e in d.entities:
    if e.dxf.layer in keeplayers:
        dxfattribs = {"layer":keeplayers[e.dxf.layer] }
        if keeplayers[e.dxf.layer] == "1":
            cuts.append(e)
        if e.dxftype() == "LINE":
            block.add_line(e.dxf.start, e.dxf.end, dxfattribs=dxfattribs)
        elif e.dxftype() == "ARC":
            block.add_arc(e.dxf.center, e.dxf.radius, e.dxf.start_angle, e.dxf.end_angle, dxfattribs=dxfattribs)
        elif e.dxftype() == "SPLINE":
            if NONURBS:
                block.add_polyline2d(e._control_points, dxfattribs=dxfattribs)
            else:
                import nurbs.Curve
                curve = nurbs.Curve.Curve()
                curve.ctrlpts = [p[:2]  for p in e.control_points]
                curve.degree = e.dxf.degree
                curve.knotvector = e.knots
                #curve.knotvector = nurbs.utilities.knotvector_autogen(curve.degree, len(curve.ctrlpts))
                #curve.weights = e.weights
                curve.delta = 0.01
                curve.evaluate_rational()
                block.add_polyline2d(curve.curvepts, dxfattribs=dxfattribs)
        else:
            print(e)
            block.add_foreign_entity(e)


In [23]:
msp = doc.modelspace()
dxfattribs = {'rotation': 0, 'layer':"block1"}
k = msp.add_blockref(selectedname, (0, 0, 0), dxfattribs=dxfattribs)
doc.set_modelspace_vport(height=2300, center=(1800, 900))
doc.saveas("test2.dxf")


In [85]:
def front(e):
    if e.dxftype() == "LINE":
        return e.dxf.start
    elif e.dxftype() == "ARC":
        return e.dxf.center + ezdxf.math.vector.Vector.from_angle(e.dxf.start_angle, e.dxf.radius)
    elif e.dxftype() == "SPLINE":
        if NONURBS:
            return ezdxf.math.vector.Vector(e._control_points[0])
    else:
        assert False

def back(e):
    if e.dxftype() == "LINE":
        return e.dxf.end
    elif e.dxftype() == "ARC":
        return e.dxf.center + ezdxf.math.vector.Vector.from_angle(e.dxf.end_angle, e.dxf.radius)
    elif e.dxftype() == "SPLINE":
        if NONURBS:
            return ezdxf.math.vector.Vector(e._control_points[-1])
    else:
        assert False

def length(e):
    if e.dxftype() == "LINE":
        return (e.dxf.end - e.dxf.start).magnitude
    elif e.dxftype() == "ARC":
        return abs(e.dxf.end_angle - e.dxf.start_angle)*e.dxf.radius
    elif e.dxftype() == "SPLINE":
        if NONURBS:
            return sum((ezdxf.math.vector.Vector(a)-ezdxf.math.vector.Vector(b)).magnitude  for a, b in zip(e._control_points, e._control_points[1:]))
    assert False
        
        
length(cuts[0])

27.272663597809704

In [66]:
for e in cuts:
    if e.dxftype() == "SPLINE":
        break
e

<class 'ezdxf.entities.spline.Spline'> SPLINE(#945)

In [74]:
ezdxf.math.vector.Vector(e._control_points[0])

Vector(-455.3990389556314, 1582.408678003407, 0.0)

In [75]:
sum((ezdxf.math.vector.Vector(a)-ezdxf.math.vector.Vector(b)).magnitude  for a, b in zip(e._control_points, e._control_points[1:]))

5334.0823718588435

In [ ]:
# this is the source of the doubling up index and then odd/even value to denote either which node (front 
# or back) of the edge, or the direction of the edge.  Recording as this way (rather than [index, bdirection] pairs) 
# means we have a good way to index into parallel table of other things, in the way that pairs cannot look things up, 
# even though this is a little less intuitive and clear. In C++ we would have done this as a struct with accessor functions
def FPSrlidat(rlends, closedist):
    rlidat = [ ]; # index*2 + (front ? 1 : 0[back])
    for i in range(len(rlends)*2):
        rlidat.append([ ]); # each will be a list of cross connections [d, j]  (these are not pairs of 
    
    var closedistSq = closedist*closedist; 
    for j in range(len(rlends)):
        rle = rlends[j]; 
        pB = rle[0] 
        jd = rle[1]*2 + (rle[2] ? 1 : 0);  # encoding the index as *2 + (forward ? 1 : 0)
        
        # loop through forwards (constrained by beyond the x-direction wider than closedist
        j1 = j+1
        while ((j1 < len(rlends) and (rlends[j1][0].x <= pB.x + closedist)):
            var rle1 = rlends[j1]; 
            var d = rle1[0] - pB;
            var dClen = d.magnitude; 
            if ((dClen <= closedist) and ((rle[1] != rle1[1]) || (rle[3] > closedist))): # { /* avoid connecting short line segment back to self (unless long enough to be a loop) */
                jd1 = rle1[1]*2 + (rle1[2] ? 1 : 0); 
                rlidat[jd].append([dCsq, jd1]); 
                rlidat[jd1].append([dCsq, jd]); 

    # sort adjacencies at endpoints by distance
    for (var j = 0; j < rlidat.length; j++) {
        rlidat[j].sort(); 
    }
    return rlidat; 
},


In [86]:
#FPSrlendsPC: function(pencutseqs, etchseqslen) // etchseqslen is the pencuts at the front that are etch numbers
def FPSrlendsPC(cuts):
    rlends = [ ]; # [ x, y, i, bfore, pathlength ]
    for i, e in enumerate(cuts):
        l = length(e)
        rlends.append([ front(e), i, False, l ]); 
        rlends.append([ back(e), i, True, l ]); 
    rlends.sort(); 
    return rlends; 




# FindClosedPathSequencesD: function(dlist, closedist)
#    // create arrays of closest links  (dlist can be padded with nulls to knock out values but retain the indexes)
#    var rlends = this.FPSrlends(dlist, dlist.length); // [ x, y, i, bfore, pathlength ]
#    var rlidat = this.FPSrlidat(rlends, dlist.length, closedist); 
#    return this.FPSjdseqs(rlidat, dlist.length); 
rlends = FPSrlendsPC(cuts)




[[Vector(-737.7426786582128, -6917.505541658997, 0.0),
  1,
  False,
  8849.275800348196],
 [Vector(-606.1595223113491, 1644.8292175142683, 0.0),
  1,
  True,
  8849.275800348196],
 [Vector(-529.4098292799999, 1521.9917914146008, 0.0),
  2,
  True,
  3328.1884408496353],
 [Vector(-515.5868449454101, 1510.245423413202, 0.0),
  0,
  True,
  27.272663597809704],
 [Vector(-515.5868449454101, 1510.245423413202, 0.0),
  3,
  True,
  748.3821626112972],
 [Vector(-489.8837985814607, 1519.364172632912, 0.0),
  0,
  False,
  27.272663597809704],
 [Vector(-462.8862289045303, 1590.7552627774205, 0.0),
  2,
  False,
  3328.1884408496353],
 [Vector(-456.7175036879598, 1563.083231413093, 0.0),
  46,
  True,
  19.370370031712643],
 [Vector(-455.3990389556314, 1582.408678003407, 0.0),
  46,
  False,
  19.370370031712643],
 [Vector(-455.3990389556314, 1582.408678003407, 0.0),
  49,
  False,
  5334.0823718588435],
 [Vector(-268.5236006620544, 803.8208947911098, 0.0),
  3,
  False,
  748.3821626112972],
 

In [47]:
type(cuts[0].dxf.start)

ezdxf.math.vector.Vector

In [49]:
cuts[0].dxf.start - ezdxf.math.vector.Vector(3000,4)

Vector(-3489.8837985814607, 1515.364172632912, 0.0)